In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pyedflib 
import pandas as pd
import mne
import sys
import eeg_lib as el

In [3]:
all_meseurments, all_targets = el.all_exp_to_array(4, [4])

In [4]:
print(type(all_meseurments))
print(type(all_meseurments))
print(all_meseurments.shape)
print(all_targets.shape)




In [5]:
import torch
import torch.nn as nn


In [6]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out


In [7]:
one_probe= all_meseurments[0]
one_probe_T= all_targets

In [8]:
one_probe[1]

In [9]:
x=[one_probe_T[0],one_probe[1000-640:1000]]



In [10]:

dataset = np.zeros((20000,640,64))
conteiner= np.zeros((640,64))

for i in range(0,len(one_probe)):
    if i > 640:
        dataset[i]= one_probe[i-640:i]



In [11]:
y_dataset= np.zeros((20000,640))
for i in range(0,len(one_probe_T)):
    if i > 640:
        y_dataset[i]= one_probe_T[i-640:i]

In [12]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import random_split
# Convert the data to PyTorch tensors
X = torch.tensor(dataset)
y = torch.tensor(y_dataset)

# Define the train and test dataset sizes
train_size = int(0.8 * len(X))
test_size = len(X) - train_size

# Split the data into train and test datasets
train_dataset, test_dataset = random_split(list(zip(X, y)), [train_size, test_size])

# Access the train and test data
X_train, y_train = zip(*train_dataset)
X_test, y_test = zip(*test_dataset)

# Convert the data to PyTorch tensors
X_train = torch.stack(X_train)
X_test = torch.stack(X_test)
y_train = torch.stack(y_train)
y_test = torch.stack(y_test)


print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


In [13]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU device is available")
else:
    device = torch.device("cpu")
    print("GPU device is not available, using CPU")


In [14]:
# Liczba elementów w tensorze
num_elements = X_train.numel()

# Rozmiar jednego elementu w bajtach (dla float32 to 4 bajty)
element_size_bytes = X_train.element_size()

# Całkowity rozmiar tensora w bajtach
total_size_bytes = num_elements * element_size_bytes

print(f"Liczba elementów: {num_elements}")
print(f"Rozmiar jednego elementu (bajty): {element_size_bytes}")
print(f"Całkowity rozmiar tensora (bajty): {total_size_bytes}")

# Opcjonalnie, przeliczenie na kilobajty (KB)
total_size_kb = total_size_bytes / 1024**2
print(f"Całkowity rozmiar tensora (MB): {total_size_kb}")

In [16]:
tensor_float16 = X_train.to(torch.float16)
# Liczba elementów w tensorze
num_elements = X_train.numel()

# Rozmiar jednego elementu w bajtach (dla float32 to 4 bajty)
element_size_bytes = X_train.element_size()

# Całkowity rozmiar tensora w bajtach
total_size_bytes = num_elements * element_size_bytes

print(f"Liczba elementów: {num_elements}")
print(f"Rozmiar jednego elementu (bajty): {element_size_bytes}")
print(f"Całkowity rozmiar tensora (bajty): {total_size_bytes}")

# Opcjonalnie, przeliczenie na kilobajty (KB)
total_size_kb = total_size_bytes / 1024**2
print(f"Całkowity rozmiar tensora (MB): {total_size_kb}")

In [ ]:
torch.cuda.empty_cache()

In [ ]:
X_train = X_train.to(tensor_float8)
y_train = y_train.to(tensor_float8)
X_test = X_test.to(tensor_float8)
y_test = y_test.to(tensor_float8)


In [ ]:
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)


In [ ]:
# Przygotowanie danych treningowych i testowych
# X_train, y_train - dane treningowe
# X_test, y_test - dane testowe

X_train= all_meseurments[0][0]
y_train

# Inicjalizacja modelu
model = LSTM(64, 64, 100,  1)

# Definicja funkcji straty i optymalizatora
criterion = nn.CrossEntropyLoss
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Pętla treningowa
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    # Ocena modelu na danych testowych
    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test)
        test_loss = criterion(test_outputs, y_test)
        predictions = (test_outputs > 0.5).float()
        accuracy = (predictions == y_test).float().mean()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Test Loss: {test_loss.item():.4f}, Accuracy: {accuracy.item():.4f}')
